In [1]:
import pandas as pd
import numpy as np
import os
import sys
import json
import requests
import gc
from tqdm import tqdm

from google.cloud import bigquery
from google.cloud.bigquery import job
from datetime import date, timedelta

PROJCECT = 'ballosodeuk'
bq = bigquery.Client(project=PROJCECT)

#  클라이언트 설정



In [2]:
query = """
    with click1 as 
  (select event_dttm, event_dt, user_id
  FROM `ballosodeuk.dw.fact_airbridge_event`, unnest(event_detail)
  where 1=1
    and event_dt between "2024-12-30" and "2025-01-09"
    and event_category in ('c__shopping_home__quick__btn (App)'))

,click2 as 
  (select event_dttm, event_dt, user_id
  FROM `ballosodeuk.dw.fact_airbridge_event`, unnest(event_detail)
  where 1=1
    and event_dt between "2024-12-30" and "2025-01-09"
    and event_category in ('c__all_income_s_money__to_exchange__btn (App)')
    )


,user_param as (
select distinct event_dt, user_id
from
  (select distinct event_dt, user_id from click1
  union all
  select distinct event_dt, user_id from click2
  ))

select a.user_id, event_dt, current_cash
from user_param a
left join ballosodeuk.dw.dim_airbridge_member b on a.user_id = b.user_id
order by user_id, event_dt

"""

df = bq.query(query).to_dataframe()



c:\Users\Owner\miniconda3\envs\pymc\lib\site-packages\google\cloud\bigquery\table.py:2309: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
c:\Users\Owner\miniconda3\envs\pymc\lib\site-packages\google\cloud\bigquery\table.py:2323: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
c:\Users\Owner\miniconda3\envs\pymc\lib\site-packages\google\cloud\bigquery\table.py:2337: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)


In [3]:
df = df.query("user_id.notnull()")
df.iloc[:,-1] = df.iloc[:,-1].fillna(0).astype(int)
df['event_dt'] = pd.to_datetime(df.event_dt.astype(str))

<ipython-input-3-90d58f8dbcc6>:2: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.iloc[:,-1] = df.iloc[:,-1].fillna(0).astype(int)


In [4]:
# 전체 유니크 유저 구하기
total_unique_users = df['user_id'].unique()

# 각 날짜별로 전체 유니크 유저가 얼마나 포함되었는지 계산
daily_coverage = pd.DataFrame()
for date in df['event_dt'].unique():
    daily_users = df[df['event_dt']==date]['user_id'].unique()
    coverage = len(np.intersect1d(daily_users, total_unique_users)) / len(total_unique_users) * 100
    daily_coverage = pd.concat([daily_coverage, 
                              pd.DataFrame({'date': [date], 'coverage_rate': [coverage]})],
                             ignore_index=True)

print("일자별 전체 유니크 유저 포함률:")
print(daily_coverage.set_index('date').sort_index())
print(f"\n평균 포함률: {daily_coverage['coverage_rate'].mean():.1f}%")

일자별 전체 유니크 유저 포함률:
            coverage_rate
date                     
2024-12-30      41.646576
2024-12-31      37.532930
2025-01-01      28.831447
2025-01-02      26.534734
2025-01-03      22.463398
2025-01-04      23.545894
2025-01-05      21.241199
2025-01-06      21.610812
2025-01-07      19.848961
2025-01-08      18.163748
2025-01-09      17.927450

평균 포함률: 25.4%


In [10]:
df_g = df.groupby('event_dt').agg({'optimal_coupon':'sum','left_cash':'sum','user_id':'count','current_cash':'sum'})
df_g['left_cash_rate'] = df_g['left_cash'] / df_g['current_cash']
df_g

,optimal_coupon,left_cash,user_id,current_cash,left_cash_rate
event_dt,,,,,
2024-12-30,600210400,430209629,52169,1030420029,0.417509
2024-12-31,547858100,391603122,47016,939461222,0.416838
2025-01-01,436887700,315308442,36116,752196142,0.419184
2025-01-02,404318500,289875088,33239,694193588,0.417571
2025-01-03,348846000,255814452,28139,604660452,0.423071
2025-01-04,377075400,279390526,29495,656465926,0.425598
2025-01-05,344803200,259205992,26608,604009192,0.429142
2025-01-06,346458400,255254053,27071,601712453,0.424213
2025-01-07,320787600,236232173,24864,557019773,0.424100


In [22]:
len(df.user_id.unique())


125266

In [130]:
import pandas as pd
import numpy as np

def find_optimal_coupon(cash, coupon_price):
    coupon_array = np.array(coupon_price)
    tg_buy_idx = np.where((cash - coupon_array) >= 0, cash - coupon_array, np.inf).argmin()
    return coupon_array[tg_buy_idx], cash - coupon_array[tg_buy_idx]

def simulate_daily_purchases(user_data, days, coupon_price, min_coupon):
    results = []
    
    # 초기 활성 사용자 설정
    active_users = user_data.copy()
    active_users['cash'] = active_users['current_cash']  # 초기 잔액 설정
    
    for day in range(1, days + 1):
        # 당일 방문 가능한 사용자 (cash가 min_coupon 이상인 사용자)
        today_users = active_users[active_users['cash'] >= min_coupon].copy()
        
        if len(today_users) > 0:
            # 쿠폰 구매 및 잔액 계산
            optimal_coupons = []
            left_cash = []
            
            for cash in today_users['cash']:
                coupon, remaining = find_optimal_coupon(cash, coupon_price)
                optimal_coupons.append(coupon)
                left_cash.append(remaining)
            
            # 사용자 잔액 업데이트
            today_users['cash'] = left_cash
            active_users.loc[today_users.index, 'cash'] = left_cash
            
            # 일일 결과 저장
            results.append({
                'day': day,
                'total_coupon_amount': sum(optimal_coupons),
                'total_remaining_cash': active_users['cash'].sum(),
                'active_users': len(active_users),
                'today_visitors': len(today_users)
            })
        else:
            # 더 이상 방문 가능한 사용자가 없는 경우
            results.append({
                'day': day,
                'total_coupon_amount': 0,
                'total_remaining_cash': active_users['cash'].sum(),
                'active_users': len(active_users),
                'today_visitors': 0
            })
            # 모든 사용자의 잔액이 min_coupon 미만이면 시뮬레이션 종료
            break
    
    return pd.DataFrame(results)

# 시뮬레이션 실행
days = 15
coupon_price = [1000, 5000, 10000, 50000]
min_coupon = min(coupon_price)

df__ = df[['user_id', 'current_cash']].drop_duplicates()
results_df = simulate_daily_purchases(df__[['user_id', 'current_cash']], days, coupon_price, min_coupon)

# 결과 출력
print("\n일별 시뮬레이션 결과:")
print(results_df.to_string(index=False))

# 추가 분석
print(f"\n시뮬레이션 종료일: {len(results_df)}일")
print(f"최종 남은 총 잔액: {results_df.iloc[-1]['total_remaining_cash']:,.0f}원")
print(f"총 쿠폰 구매액: {results_df['total_coupon_amount'].sum():,.0f}원")


일별 시뮬레이션 결과:
 day  total_coupon_amount  total_remaining_cash  active_users  today_visitors
   1           1114862000            1131427196        125266          117489
   2            531767000             599660196        125266          102940
   3            307783000             291877196        125266           83152
   4            151081000             140796196        125266           60427
   5             49788000              91008196        125266           36817
   6             18920000              72088196        125266           18683
   7              8556000              63532196        125266            8467
   8              2883000              60649196        125266            2870
   9               512000              60137196        125266             503
  10                 9000              60128196        125266               5
  11                 1000              60127196        125266               1
  12                 1000              60126196   

In [30]:
import pandas as pd
import numpy as np

def find_optimal_coupon(cash, coupon_price):
   coupon_array = np.array(coupon_price)  
   tg_buy_idx = np.where((cash - coupon_array) >= 0, cash - coupon_array, np.inf).argmin()
   return coupon_array[tg_buy_idx], cash - coupon_array[tg_buy_idx]

def simulate_daily_purchases(user_data, days, coupon_price, min_coupon, daily_cap):
   results = []
   
   # 초기 활성 사용자 설정
   active_users = user_data.copy()
   active_users['cash'] = active_users['current_cash']  # 초기 잔액 설정
   
   for day in range(1, days + 1):
       # 당일 방문 가능한 사용자 (cash가 min_coupon 이상인 사용자)
       potential_users = active_users[active_users['cash'] >= min_coupon]
       
       if len(potential_users) > 0:
           # 일일 방문자 수를 daily_cap으로 제한
           n_visitors = min(len(potential_users), daily_cap)
           # 랜덤 샘플링으로 today_users 선택
           today_users = potential_users.sample(n=n_visitors).copy()
           
           # 쿠폰 구매 및 잔액 계산
           optimal_coupons = []
           left_cash = []
           
           for cash in today_users['cash']:
               coupon, remaining = find_optimal_coupon(cash, coupon_price)
               optimal_coupons.append(coupon)
               left_cash.append(remaining)
           
           # 사용자 잔액 업데이트
           today_users['cash'] = left_cash
           active_users.loc[today_users.index, 'cash'] = left_cash
           
           # 일일 결과 저장
           results.append({
               'day': day,
               'total_coupon_amount': sum(optimal_coupons),
               'total_remaining_cash': active_users['cash'].sum(),
               'active_users': len(active_users),
               'today_visitors': len(today_users),
               'potential_visitors': len(potential_users)
           })
       else:
           # 더 이상 방문 가능한 사용자가 없는 경우
           results.append({
               'day': day,
               'total_coupon_amount': 0,
               'total_remaining_cash': active_users['cash'].sum(),
               'active_users': len(active_users),
               'today_visitors': 0,
               'potential_visitors': 0
           })
           # 모든 사용자의 잔액이 min_coupon 미만이면 시뮬레이션 종료
           break
   
   return pd.DataFrame(results)

# 시뮬레이션 실행
days = 100
coupon_price = [200,600,2000,6000,10000,20000]
min_coupon = min(coupon_price)
daily_cap = 22000

df__ = df[['user_id', 'current_cash']].drop_duplicates()
results_df = simulate_daily_purchases(df__[['user_id', 'current_cash']], days, coupon_price, min_coupon, daily_cap)

# 결과 출력
print("\n일별 시뮬레이션 결과:")
print(results_df.to_string(index=False))

# 추가 분석
print(f"\n시뮬레이션 종료일: {len(results_df)}일")
print(f"최종 남은 총 잔액: {results_df.iloc[-1]['total_remaining_cash']:,.0f}원")
print(f"총 쿠폰 구매액: {results_df['total_coupon_amount'].sum():,.0f}원")

pd.set_option('display.max_rows', None) 
pd.DataFrame(results_df)




일별 시뮬레이션 결과:
 day  total_coupon_amount  total_remaining_cash  active_users  today_visitors  potential_visitors
   1            242150400            2004138796        125266           22000              121274
   2            219826600            1784312196        125266           22000              120260
   3            198870800            1585441396        125266           22000              118934
   4            180684000            1404757396        125266           22000              117297
   5            163762800            1240994596        125266           22000              115333
   6            148160800            1092833796        125266           22000              113134
   7            134596000             958237796        125266           22000              110488
   8            120996400             837241396        125266           22000              107531
   9            109133600             728107796        125266           22000              104166
  10  

,day,total_coupon_amount,total_remaining_cash,active_users,today_visitors,potential_visitors
0,1,242150400,2004138796,125266,22000,121274
1,2,219826600,1784312196,125266,22000,120260
2,3,198870800,1585441396,125266,22000,118934
3,4,180684000,1404757396,125266,22000,117297
4,5,163762800,1240994596,125266,22000,115333
5,6,148160800,1092833796,125266,22000,113134
6,7,134596000,958237796,125266,22000,110488
7,8,120996400,837241396,125266,22000,107531
8,9,109133600,728107796,125266,22000,104166
9,10,99474800,628632996,125266,22000,100409


In [132]:
df__ = df[['user_id', 'current_cash']].drop_duplicates()

In [12]:
df[['user_id', 'current_cash']].drop_duplicates().current_cash.sum()


2246289196

퍼널

In [31]:
query = """with stack as 
  (select user_id, register_dt, sum(amt) as amt
  from
    (select m.wk_id as user_id, register_dt, amt 
    from ballosodeuk.dw.fact_shopby_reward r
    left join ballosodeuk.dw.dim_shopby_member m
      on r.member_no = m.member_no
    where 1=1
      and accumulation_status = "지급"
      and register_dt >= "2024-12-30"
      and reason_detail in ("쇼핑지원금 교환권 적립","쇼핑지원금 상품권 적립","쇼핑지원금 전환","쇼핑지원금 5,000원 교환 쿠폰" ))
  group by user_id, register_dt)

  ,click1 as 
  (select event_dttm, event_dt, user_id
  FROM `ballosodeuk.dw.fact_airbridge_event`, unnest(event_detail)
  where 1=1
    and event_dt between "2024-12-30" and "2025-01-09"
    and event_category in ('c__shopping_home__quick__btn (App)'))

,click2 as 
  (select event_dttm, event_dt, user_id
  FROM `ballosodeuk.dw.fact_airbridge_event`, unnest(event_detail)
  where 1=1
    and event_dt between "2024-12-30" and "2025-01-09"
    and event_category in ('c__all_income_s_money__to_exchange__btn (App)')
    )

,user_param as (
select distinct event_dt, user_id
from
  (select distinct event_dt, user_id from click1
  union all
  select distinct event_dt, user_id from click2
  ))

,exchange as 
  (
    select register_dttm, user_id, event, sum(amt) as amt
    from 
      (
        select register_dttm, wk_id as user_id, 
          case 
            when reason_detail in ('쇼핑지원금 교환권 적립', '쇼핑지원금 상품권 적립', '쇼핑지원금 5,000원 교환 쿠폰') then "교환권"
            else  "보상" end as event,
            amt
        from ballosodeuk.dw.fact_shopby_reward a
        left join `ballosodeuk.dw.dim_shopby_member` b on a.member_no = b.member_no
        where 1=1
          and accumulation_status = '지급'
          and register_dt >= "2024-12-30"
          and wk_id in (
            select user_id from user_param)
      )
    group by register_dttm, event, user_id
  )

,tg as 
  (select order_dttm,order_dt, order_no, member_no, first_pay_amt, first_sub_pay_amt, claim_status_type
  from
    (select order_dttm, order_dt, order_no, member_no, first_pay_amt, first_sub_pay_amt,  claim_status_type
    from ballosodeuk.dw.fact_shopby_order, unnest(product_option)
    where length(claim_status_type) < 1
    )
  group by order_dttm,order_dt, order_no, member_no, first_pay_amt, first_sub_pay_amt, claim_status_type)

,tgs as
  (select order_dttm as register_dttm, user_id, "구매" as event , first_pay_amt as amt
  from
    (
      select wk_id as user_id, min(order_dt) over (partition by a.member_no) as first_purchase, * except (member_no)
      from tg a
      left join ballosodeuk.dw.dim_shopby_member b on a.member_no = b.member_no
      where order_dt >= "2024-12-30"
    )
  -- where first_purchase >= "2024-12-30"
)

,merged as 
  (select * from exchange
  union all 
  select * from tgs)

,funnel as 
  (select register_dttm, user_id
    ,funnel_1
    ,case when funnel_1 = "교환권" then funnel_1_dt end as funnel_1_dt
    ,case when funnel_2 = "구매" and funnel_2_dt > funnel_1_dt then funnel_2 end as funnel_2
    ,case when funnel_2 = "구매" and funnel_2_dt > funnel_1_dt then funnel_2_dt end as funnel_2_dt
    ,case when 
      funnel_2 = "구매" 
      and funnel_2_dt > funnel_1_dt
      and funnel_3 = "보상" 
      and funnel_3_dt > funnel_2_dt
      then funnel_3 end as funnel_3
    ,case when 
      funnel_2 = "구매" 
      and funnel_2_dt > funnel_1_dt
      and funnel_3 = "보상" 
      and funnel_3_dt > funnel_2_dt
      then funnel_3_dt end as funnel_3_dt
  from
    (select register_dttm,user_id, event as funnel_1, register_dttm as funnel_1_dt
      ,lead(event) over (partition by user_id order by register_dttm) as funnel_2
      ,lead(register_dttm) over (partition by user_id order by register_dttm) as funnel_2_dt
      ,lead(event, 2) over (partition by user_id order by register_dttm) as funnel_3
      ,lead(register_dttm, 2) over (partition by user_id order by register_dttm) as funnel_3_dt
    from merged)
  where 1=1
    and funnel_1 = "교환권")

select * from funnel"""

In [32]:
ff = bq.query(query).to_dataframe()

c:\Users\Owner\miniconda3\envs\pymc\lib\site-packages\google\cloud\bigquery\table.py:2309: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
c:\Users\Owner\miniconda3\envs\pymc\lib\site-packages\google\cloud\bigquery\table.py:2323: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
c:\Users\Owner\miniconda3\envs\pymc\lib\site-packages\google\cloud\bigquery\table.py:2337: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)


In [4]:
df

,user_id,event_dt,current_cash
10,000087ef-6929-4532-85be-33661b2082d5,2025-01-06,25905
11,0000d355-7050-4a3e-90d1-2aa0b1198aba,2024-12-30,8636
12,0000d355-7050-4a3e-90d1-2aa0b1198aba,2024-12-31,8636
13,00022901-aed3-4327-bb0a-29bbdc978dfa,2024-12-30,18658
14,000256cd-ebbd-46ac-94e9-93a2b1ace57d,2025-01-05,7283
...,...,...,...
349932,fffff0da-172e-421f-87be-12324d714cbd,2025-01-04,14865
349933,fffff0da-172e-421f-87be-12324d714cbd,2025-01-06,14865
349934,fffff0da-172e-421f-87be-12324d714cbd,2025-01-07,14865
349935,fffff0da-172e-421f-87be-12324d714cbd,2025-01-08,14865


In [23]:
df = df.groupby('user_id').agg({'current_cash':'first'})
df = df.reset_index()

In [27]:
bins = [
    0,
    200,
    600,
    1000,
    2000,
    6000,
    10000,
    20000,
    float('inf')
]

labels = [
    "0 ~ 200",
    "200 ~ 600", 
    "600 ~ 1,000",
    "1,000 ~ 2,000",
    "2,000 ~ 6,000",
    "6,000 ~ 10,000",
    "10,000 ~ 20,000",
    "20,000 ~"
]

df['cash_bin'] = pd.cut(df['current_cash'], bins=bins, labels=labels)
df_ = df.groupby('cash_bin').agg({'user_id': lambda x: len(set(x)), 'current_cash': 'sum'})
df_ = df_.reindex(labels)
df_


,user_id,current_cash
cash_bin,,
0 ~ 200,2914,234471
200 ~ 600,2011,788474
"600 ~ 1,000",1765,1417268
"1,000 ~ 2,000",4942,7456447
"2,000 ~ 6,000",21810,86727898
"6,000 ~ 10,000",17476,137782372
"10,000 ~ 20,000",28387,411599465
"20,000 ~",44824,1596926546


In [25]:
df_.current_cash.sum()

2242932941